In [1]:
%matplotlib inline
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, graphviz_layout
from pyomo.environ import ConcreteModel, SolverFactory, TerminationCondition, \
    value, Var, Constraint, Expression, Objective, TransformationFactory, units as pyunits
from pyomo.network import Arc, SequentialDecomposition
from idaes.core.util.model_statistics import degrees_of_freedom

### TODO: CREATE INTRO

##### Import WaterTAP3 Package

In [2]:
import watertap as wt

##### Step 1: Look up unit process library. Returns a List.

In [3]:
wt.unit_process_library_list

['chlorination_twb',
 'media_filtration_twb',
 'microfiltration_twb',
 'ultrafiltration_twb',
 'nanofiltration_twb',
 'coag_and_flocro_twb',
 'uv_twb',
 'ro_bor',
 'uvozone_twb',
 'mbr',
 'water_pumping_station',
 'ro_deep',
 'media_filtration',
 'coag_and_floc',
 'lime_softening',
 'ro_deep',
 'treated_storage_24_hr',
 'sedimentation',
 'water_pumping_station',
 'sulfuric_acid_addition',
 'sodium_bisulfite_addition',
 'co2_addition',
 'ammonia_addition',
 'municipal_drinking',
 'sw_onshore_intake',
 'holding_tank',
 'tri_media_filtration',
 'cartridge_filtration']

##### Step 3: Set up IDAES flowsheet for watertap. This selects a property package for you (see water_props.py). The property package could be changed in the future for performing heat/energy and other balances.

In [4]:
m = wt.watertap_setup(dynamic = False)

##### Step 3: Add a water source and define inlet flow.

In [5]:
m = wt.design.add_water_source(m = m, source_name = "source1", link_to = None, 
                     reference = "Poseidon", water_type = "Wastewater", 
                     case_study = "Typical untreated domestic wastewater",
                               flow = 4.5833) # m3/s (4.38 m3/s = 100 MGD) (16500 m3/h = 104.6121 MGD = 4.5833 m3/s)

##### Step 4: Add a unit process. You can also add a unit process a link it directly to another unit process or source.

In [6]:
m = wt.design.add_unit_process(m = m, unit_process_name = "ro", unit_process_type = 'treated_storage_24_hr')

##### Step X: Connect the unit process to the source.

In [7]:
m = wt.design.connect_blocks(m = m, 
                    stream_name = "arc1",
                    from_node = "source1",
                    to_node = "ro")

In [8]:
wt.display.show_train2(model_name=m)

##### Step 5:  Run model and display results. The resutls display will be put into easier format to read for user soon.

In [9]:
# Set up a solver in Pyomo and solve
solver1 = SolverFactory('ipopt')
results = solver1.solve(m, tee=True)

# Transform Arc to construct linking equations
TransformationFactory("network.expand_arcs").apply_to(m)
seq = SequentialDecomposition()
G = seq.create_graph(m)
print("degrees_of_freedom:", degrees_of_freedom(m))

# Display the inlets and outlets of each unit
for node in G.nodes():
    print("----------------------------------------------------------------------")
    print(node)
    
    if "split" in (str(node).replace('fs.', '')): 
        getattr(m.fs, str(node).replace('fs.', '')).inlet.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet1.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet2.display()
    elif "use" in (str(node).replace('fs.', '')): 
        getattr(m.fs, str(node).replace('fs.', '')).inlet.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet.display()
    elif "mixer" in (str(node).replace('fs.', '')): 
        getattr(m.fs, str(node).replace('fs.', '')).inlet1.display()
        getattr(m.fs, str(node).replace('fs.', '')).inlet2.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet.display()
    else:
        getattr(m.fs, str(node).replace('fs.', '')).inlet.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet.display()
        getattr(m.fs, str(node).replace('fs.', '')).waste.display()

        
    print("Show some costing values")
    print("---------------------")
    
    if "source" in (str(node).replace('fs.', '')): 
        print("should skip:", (str(node).replace('fs.', '')))
        continue
    elif "use" in (str(node).replace('fs.', '')): 
        print("should skip:", (str(node).replace('fs.', '')))
        continue
    elif "split" in (str(node).replace('fs.', '')): 
        print("should skip:", (str(node).replace('fs.', '')))
        continue  
    elif "mixer" in (str(node).replace('fs.', '')): 
        print("should skip:", (str(node).replace('fs.', '')))
        continue
    else:
        print("should have a cost", (str(node).replace('fs.', '')))
        if getattr(m.fs, str(node).replace('fs.', '')).costing.fixed_cap_inv_unadjusted() is not None:
            print("total_up_cost:" , 
                  getattr(m.fs, str(node).replace('fs.', '')).costing.fixed_cap_inv_unadjusted())
                
        else:
            getattr(m.fs, str(node).replace('fs.', '')).costing.fixed_cap_inv_unadjusted.display()
    
    print("----------------------------------------------------------------------")
   

Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       74
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:       30
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

In [10]:
####TO DO LOAD AND SAVE!!

In [11]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [12]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [13]:
# wt.display.show_train(TT)